# Chapter 02 - Data and Sampling Distributions

## Random Sampling and Sample Bias

*Random sampling* is the process of drawing a set of elements (or individuals) from a larger set (called population) in a random way. Random sampling can be done *with replacement*, where each sampled element returns to the population after being sampled for possible future selection. Or else, it can be done *without replacement*, which implies that the sampled element does not return to the population once selected.

In stratified sampling, the population is divided into groups (strata), and random samples are taken from each group (stratum).

**Size vs Quality: When does size matter?**

In most scenarios, data quality is more important than data quantity, and random sampling can help reducing bias and facilitate quality improvement. However, **when data is very sparse**, sampling won't help as much. In those cases, having a large amount of data may be even more important.  

## Selection Bias

*Selection bias* refers to the practice of selectively choosing data - consciously or unconsciouly - in a way that leads to a conclusion that is misleading of ephemeral.

Keep in mind the following paragraph copied directly from the book:

"If you specify a hypothesis and conduct a well-designed experiment to test it, you can have high confidence in the conclusion. This is frequently not what occurs, however. Often, one looks at available data and tries to discern patterns. But are the patterns real? Or are they just the product of data snooping—that is, extensive hunting through the data until something interesting emerges? There is a saying among statisticians: 'If you torture the data long enough, sooner or later it will confess.'"

**The correct procedure to avoid selection bias is to specify a hypothesis and then collecting data following randomization and random sampling principles ensures against bias.**

All other forms of data analysis run the risk of selection bias (through repeated running models until you find something "interesting", data snooping in research, and after-the-fact selection of interesting events - which happens when you first run a research on the dataset before specifying a hypothesis.)